In [ ]:
!pip install timm

In [ ]:
model_save_path = 'model.pth'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/train.zip -d /content/

In [ ]:
!unzip /content/drive/MyDrive/test.zip -d /content/

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

In [ ]:
from sklearn.metrics import f1_score
from glob import glob
import pathlib
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pprint import pprint
import urllib.request
import csv
import numpy as np
from torch.cuda import amp
from tqdm import tqdm
import time
import copy
from collections import defaultdict
from sklearn.metrics import mean_squared_error
import joblib
import gc
import os
from sklearn.model_selection import train_test_split
import gc
import cv2
import copy
import time
import random
from PIL import Image


# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import random
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.cuda import amp

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

import json

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import timm
from sklearn.metrics import f1_score

In [ ]:
#pip install wandb

In [ ]:
#!wandb login

In [ ]:
#import wandb

#wandb.init(project="my-test-project")

In [ ]:
config = {'batch':8,
        'path': './train/train'}

# TODO : ToTensorV2
albumentations_transform = A.Compose([
        
A.Resize(224,224),
A.HorizontalFlip(p=0.5),
A.RandomBrightnessContrast(p=0.2),
ToTensorV2(p =1.0)
], p=1.0)


In [ ]:
train_json = sorted(glob('train/*/*.json'))

labels = []
crops = []
diseases = []
risks = []
for i in range(len(train_json)):
    with open(train_json[i], 'r') as f:
        sample = json.load(f)
        crop = sample['annotations']['crop']
        disease = sample['annotations']['disease']
        risk = sample['annotations']['risk']
        label=f"{crop}_{disease}_{risk}"
        crops.append(crop)
        diseases.append(disease)
        risks.append(risk)
        labels.append(label)

label_encoder = sorted(np.unique(labels))
label_encoder = {key:value for key,value in zip(label_encoder, range(len(label_encoder)))}

label1 = [label_encoder[k] for k in labels]
label2 = {}
for key, value in label_encoder.items():
    label2[key] = value
label_decoder = {val:key for key, val in label_encoder.items()}

# label_encoder가 decoder처럼 vlaue -> key

print(label_decoder)


In [ ]:
transform = A.Compose([
    A.Resize(224, 224),
], p=1.)

In [ ]:
img_path =  sorted(glob('train/*/*.jpg'))
label_path = sorted(glob('train/*/*.json'))
class PlantImageDataset(Dataset):
    def __init__(self, img_path, label_path, transform=None):
        self.img_labels = labels
        self.transform = transform
        self.img_path = img_path
        self.label_path = label_path
        
    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        path = os.path.join(self.img_path[idx])
      
        image = cv2.imread(path)
        
        label_path = self.label_path[idx]
        # TODO :숫자 반환 
        with open(label_path, 'r') as f:
          sample = json.load(f)
          crop = sample['annotations']['crop']
          disease = sample['annotations']['disease']
          risk = sample['annotations']['risk']
          label=f"{crop}_{disease}_{risk}"

          label= label_encoder[label]
          

          if self.transform:
            image = self.transform(image=image)['image']
        
        return image, label

dataset = PlantImageDataset(img_path = img_path, label_path = label_path, transform=albumentations_transform)
train_dataloader = DataLoader(dataset, batch_size = config['batch'], shuffle = True)
valid_dataset = PlantImageDataset(img_path = img_path, label_path = label_path, transform=transform)
vaild_dataloader = DataLoader(valid_dataset, batch_size = config['batch'], shuffle = True)

In [ ]:
# Albumentations for augmentations
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Augmentations
transform = A.Compose([
    A.Resize(224, 224),
], p=1.)

fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(15, 15))

for x, _ in dataset:

    img = x
    batch_size = 64
    img = np.array(img).T
    print(type(img))
    print(img.shape)
    for i, ax in zip(range(batch_size), axes.flatten()):
        ax.imshow(img)
        break
    break

In [ ]:
from torchvision.models import resnet18

backbone = 'resnet18'
num_classes = 25
emberdder = 512
device = "cuda" if torch.cuda.is_available() else "cpu"

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model = timm.create_model(backbone, pretrained=True, num_classes = num_classes)      
    
    def forward(self, images):
        images = self.model(images)
        return images

model = Model().to(device)

In [ ]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# class SwinNet(nn.Module):
#     def __init__(self, model_name='swin_small_patch4_window7_224', out_features=25,
#                  inp_channels=3, pretrained=True):
#         super().__init__()
#         self.model = timm.create_model(model_name, pretrained=pretrained,
#                                        in_chans=inp_channels)
#         n_features = self.model.head.in_features
#         self.model.head = nn.Linear(n_features, out_features, bias=True)
        
#     def forward(self, x):
#         return self.model(x)

# model = SwinNet().to(device)

In [ ]:
learning_late = 1e-5
optimizer = torch.optim.Adam(model.parameters(), lr=learning_late)

In [ ]:
def accuracy_function(real, pred):
    score = f1_score(real, pred, average='macro')
    return score

In [ ]:
def train_one_epoch(model, optimizer, train_dataloader, device, epoch):
    model.train()
   # wandb.init(project='test', entity = 'suho')
    #wandb.watch(model, optimizer, log="all", log_freq=10)

    train_loss = 0
    train_pred=[]
    train_target=[]
    best = 0
    

    for batch in tqdm(train_dataloader):
      optimizer.zero_grad()
      img = torch.tensor(batch[0], dtype=torch.float32, device=device)
      target = torch.tensor(batch[1], dtype=torch.long, device=device)
        
      logits = model(img.float()/256).to(device)

      loss_fn = nn.CrossEntropyLoss()
      loss = loss_fn(logits,target)
      loss.backward()
 
      optimizer.step()
      train_loss += loss.item()/len(train_dataloader)
      train_pred += logits.argmax(1).detach().cpu().numpy().tolist()
      train_target += target.detach().cpu().numpy().tolist()

    train_f1 = accuracy_function(train_target, train_pred)

    print(f"loss : {train_loss:7f}, f1 : {train_f1:7f}") #acc: {acc:7f}
    if train_f1 >= best:
      best = train_f1
      torch.save(model.state_dict(), 'model.pth')
  
    return loss, train_f1
    

In [ ]:
@torch.no_grad()
def valid_one_epoch(model, vaild_dataloader, device, epoch):
    model.eval()
    
    vaild_loss = 0
    vaild_pred=[]
    vaild_target=[]
    best = 0
    

    for batch in tqdm(vaild_dataloader):
      img = torch.tensor(batch[0], dtype=torch.float32, device=device)
      target = torch.tensor(batch[1], dtype=torch.long, device=device)
        
      logits = model(img.float()/256).to(device)

      loss_fn = nn.CrossEntropyLoss()
      loss = loss_fn(logits,target)
      loss.backward()

      vaild_loss += loss.item()/len(vaild_dataloader)
      vaild_pred += logits.argmax(1).detach().cpu().numpy().tolist()
      vaild_target += target.detach().cpu().numpy().tolist()

    vaild_f1 = accuracy_function(vaild_target, vaild_pred)

    print(f"loss : {vaild_loss:7f}, f1 : {vaild_f1:7f}") #acc: {acc:7f}
  
    return vaild_loss, vaild_f1

In [ ]:
epochs = 10
for epoch in range(epochs):

    print(f"Epoch {epoch+1}\n-------------------------------")
    train_one_epoch(model, optimizer, train_dataloader, device, epoch)
    valid_one_epoch(model, vaild_dataloader, device, epoch)


In [ ]:
test_jpg = sorted(glob('test/*/*.jpg'))

In [ ]:
transform = A.Compose([
    A.Resize(224, 224),
    ToTensorV2(p =1.0)
], p=1.)


In [ ]:
class custom_dataset(Dataset):
    def __init__(self, img_path, transform = None, mode='test'):
        self.mode=mode
        self.transform = transform
        self.img_path = img_path
    def __len__(self):
        return len(self.img_path)
    def __getitem__(self, idx):
        path = os.path.join(self.img_path[idx])
        image = cv2.imread(path)
        if self.transform:
            image = self.transform(image=image)['image']
        return image

test_dataset = custom_dataset(test_jpg, transform = transform, mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=1)
len(test_loader)

In [ ]:
model.load_state_dict(torch.load(model_save_path))

model.eval()
test_preds=[]
with torch.no_grad():
    for batch in tqdm(test_loader):
      batch[0] = batch[0]
      img = torch.tensor(batch[0], device=device)
      img = img.unsqueeze(0)

      with torch.cuda.amp.autocast():
          pred = model(img.float()/256)
      test_preds += pred.argmax(1).detach().cpu().numpy().tolist()


In [ ]:
test_pred = [str(label_decoder[i]) for i in test_preds]
len(test_pred)

In [ ]:
submission_template = pd.read_csv('sample_submission.csv')
submission_template.head()

In [ ]:
submission = submission_template.copy()
submission['label'] = test_pred
print(len(submission))
submission.head()

In [ ]:
submission_file_name = 'f1f1.csv'
submission.to_csv(submission_file_name, index=False)